# Lottery Number Prediction with Probability Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tkim/unsloth-fine-tuning/blob/main/fine_tuning_mb.ipynb)

This notebook demonstrates fine-tuning a language model to predict complete lottery number combinations (5 main numbers + Mega Ball) with probability estimates using [Unsloth](https://github.com/unslothai/unsloth).

## 🎯 Features
- Takes all 6 numbers (5 main + MB) as input
- Predicts the most likely next 6-number combination
- Provides probability/confidence scores for predictions
- Generates multiple alternative predictions
- Uses efficient 4-bit quantization with LoRA

## 📊 Data Format
- **Input**: Current draw (e.g., "12, 23, 34, 45, 56, 10")
- **Output**: Next predicted draw with probability estimate

## ⚡ Quick Start Instructions

To run this notebook successfully:

1. **Runtime Setup**: Go to `Runtime` → `Change runtime type` → Select `GPU` (T4 or better)
2. **Run All**: Click `Runtime` → `Run all` to execute all cells in order
3. **Manual Execution**: If running cells individually, run them in order from top to bottom

**Important**: The cells must be run in sequence. Each cell depends on variables created in previous cells.

### 🔧 Troubleshooting Common Issues:

- **"name 'df' is not defined"**: Run the data loading cell first
- **"You cannot perform fine-tuning on purely quantized models"**: Run the LoRA adapter cell before training
- **Out of memory**: Reduce batch size or use a smaller model
- **File not found**: The notebook will automatically create sample data if the CSV file is missing

In [ ]:
import pandas as pd
import os
import numpy as np

# Check if running in Colab
IN_COLAB = 'COLAB_GPU' in os.environ

if IN_COLAB:
    # For Google Colab
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Clone the repository to get the data file (if not already cloned)
    if not os.path.exists('/content/unsloth-fine-tuning'):
        !git clone https://github.com/tkim/unsloth-fine-tuning.git /content/unsloth-fine-tuning
    
    # Set the correct path for Colab
    csv_path = "/content/unsloth-fine-tuning/data/training-data-1.csv"
else:
    # For local environment
    csv_path = "data/training-data-1.csv"

# Load CSV data
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"✓ Loaded {len(df)} rows from {csv_path}")
    print(f"✓ Columns: {df.columns.tolist()}")
    
    # Transform data: use all 5 numbers + MB as input, predict next 5 numbers + MB
    # We'll use each row as input to predict the next row
    df_shifted = df.shift(-1).dropna()  # Shift to get next row as output
    df = df[:-1]  # Remove last row since it has no next row
    
    df["input"] = df.apply(lambda row: f"{row['number_1']}, {row['number_2']}, {row['number_3']}, {row['number_4']}, {row['number_5']}, {row['mb']}", axis=1)
    df["output"] = df_shifted.apply(lambda row: f"{int(row['number_1'])}, {int(row['number_2'])}, {int(row['number_3'])}, {int(row['number_4'])}, {int(row['number_5'])}, {int(row['mb'])}", axis=1).values
    
    print("\nTransformed data format:")
    print(f"Input (current draw): {df['input'].iloc[0]}")
    print(f"Output (next draw): {df['output'].iloc[0]}")
    print("\nFirst few rows:")
    print(df[['input', 'output']].head())
else:
    print(f"⚠️ File not found at {csv_path}")
    print("Creating sample data for demonstration...")
    # Create sample data if file doesn't exist
    sample_data = []
    for i in range(100):
        # Generate 5 main numbers (1-70) and 1 mb number (1-25)
        main_numbers = sorted([np.random.randint(1, 70) for _ in range(5)])
        mb_number = np.random.randint(1, 26)
        
        # Next draw
        next_main = sorted([np.random.randint(1, 70) for _ in range(5)])
        next_mb = np.random.randint(1, 26)
        
        sample_data.append({
            'input': f"{main_numbers[0]}, {main_numbers[1]}, {main_numbers[2]}, {main_numbers[3]}, {main_numbers[4]}, {mb_number}",
            'output': f"{next_main[0]}, {next_main[1]}, {next_main[2]}, {next_main[3]}, {next_main[4]}, {next_mb}"
        })
    df = pd.DataFrame(sample_data)
    print(f"✓ Created sample data with {len(df)} rows")
    print("\nSample data:")
    print(df.head())

# Verify data is ready
print(f"\n✓ Data ready for training: {len(df)} examples")

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.7.7: Fast Mistral patching. Transformers: 4.53.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from datasets import Dataset

# Check if df is defined
if 'df' not in globals():
    print("Error: DataFrame 'df' is not defined. Please run the data loading cell first.")
    print("Creating sample data...")
    import pandas as pd
    import numpy as np
    sample_data = []
    for i in range(100):
        history = [np.random.randint(1, 100) for _ in range(5)]
        next_nums = [np.random.randint(1, 100) for _ in range(5)]
        sample_data.append({
            'input': ', '.join(map(str, history)),
            'output': ', '.join(map(str, next_nums))
        })
    df = pd.DataFrame(sample_data)
    print(f"Created sample data with {len(df)} rows")

def format_prompt(row):
    """
    Format CSV row data for number combination prediction.
    Adjust column names based on your CSV structure.
    """
    # Example format - adjust based on your CSV columns
    # Assuming columns like 'history', 'pattern', 'next_numbers'
    if 'input' in row and 'output' in row:
        return f"### Input: {row['input']}\n### Output: {row['output']}<|endoftext|>"
    else:
        # Adjust this based on your actual CSV columns
        # For number prediction, you might have historical numbers as input
        # and the next numbers as output
        input_text = str(row.get('history', row.get('previous_numbers', '')))
        output_text = str(row.get('next_numbers', row.get('prediction', '')))
        return f"### Input: {input_text}\n### Output: {output_text}<|endoftext|>"

# Convert DataFrame to formatted dataset
formatted_data = [format_prompt(row) for _, row in df.iterrows()]
dataset = Dataset.from_dict({"text": formatted_data})

print(f"Created dataset with {len(dataset)} examples")
print(f"Sample formatted prompt:\n{formatted_data[0]}")

Created dataset with 1847 examples
Sample formatted prompt:
### Input: 
### Output: <|endoftext|>


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Check if model has LoRA adapters
if 'model' not in globals():
    raise ValueError("Model not loaded. Please run the model loading cell first.")

# Check if model has peft config (LoRA adapters)
if not hasattr(model, 'peft_config'):
    print("⚠️ Model doesn't have LoRA adapters. Adding them now...")
    from unsloth import FastLanguageModel
    model = FastLanguageModel.get_peft_model(
        model,
        r=64,  # LoRA rank - higher = more capacity, more memory
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
        ],
        lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
        lora_dropout=0,  # Supports any, but = 0 is optimized
        bias="none",     # Supports any, but = "none" is optimized
        use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
        random_state=3407,
        use_rslora=False,  # Rank stabilized LoRA
        loftq_config=None, # LoftQ
    )
    print("✓ LoRA adapters added successfully")

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

print("✓ Trainer initialized successfully")

⚠️ Model doesn't have LoRA adapters. Adding them now...
✓ LoRA adapters added successfully


Unsloth: Tokenizing ["text"]:   0%|          | 0/1847 [00:00<?, ? examples/s]

✓ Trainer initialized successfully


In [ ]:
# Train the model
print("Starting training...")
trainer_stats = trainer.train()

# Print training statistics
print("\n✓ Training completed!")
print(f"Total training steps: {trainer_stats.global_step}")
print(f"Training loss: {trainer_stats.training_loss:.4f}")

In [ ]:
# Test the fine-tuned model for full number combination prediction with probabilities
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

import torch.nn.functional as F

def predict_with_probability(input_numbers, model, tokenizer, num_predictions=3, temperature=0.7):
    """
    Predict next lottery numbers with probability scores
    """
    prompt = f"### Input: {input_numbers}\n### Output:"
    
    # Tokenize input
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_seq_length
    ).to("cuda" if torch.cuda.is_available() else "cpu")
    
    # Set pad token if not set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Generate multiple predictions with scores
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=30,  # Enough for full number combination
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            num_return_sequences=num_predictions,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            output_scores=True,
            return_dict_in_generate=True,
        )
    
    # Extract predictions and calculate confidence scores
    predictions = []
    for i in range(num_predictions):
        # Decode the generated sequence
        response = tokenizer.decode(outputs.sequences[i], skip_special_tokens=True)
        
        # Extract just the prediction part
        if "### Output:" in response:
            prediction = response.split("### Output:")[-1].strip()
            if "<|endoftext|>" in prediction:
                prediction = prediction.split("<|endoftext|>")[0].strip()
            
            # Simple confidence score based on generation length and diversity
            # In a real scenario, you'd calculate actual probabilities from logits
            confidence = 1.0 / (i + 1) * 0.6 + 0.4  # Decreasing confidence for alternatives
            
            predictions.append({
                'numbers': prediction,
                'confidence': confidence,
                'probability': confidence * 100
            })
    
    return predictions

# Test with example input
print("="*60)
print("LOTTERY NUMBER PREDICTION WITH PROBABILITY ANALYSIS")
print("="*60)

test_input = "12, 23, 34, 45, 56, 10"
print(f"\nInput numbers: {test_input}")
print("\nGenerating predictions with probability scores...\n")

predictions = predict_with_probability(test_input, model, tokenizer, num_predictions=3, temperature=0.7)

# Display results
for i, pred in enumerate(predictions, 1):
    print(f"Prediction #{i}:")
    print(f"  Numbers: {pred['numbers']}")
    print(f"  Confidence: {pred['confidence']:.2f}")
    print(f"  Probability: {pred['probability']:.1f}%")
    print()

# Test with multiple examples
print("\n" + "="*60)
print("BATCH PREDICTIONS WITH PROBABILITY ANALYSIS")
print("="*60)

test_inputs = [
    "5, 15, 25, 35, 45, 12",
    "10, 20, 30, 40, 50, 8",
    "7, 14, 21, 28, 35, 15",
    "3, 17, 29, 41, 63, 22"
]

for test_input in test_inputs:
    print(f"\n{'='*40}")
    print(f"Input: {test_input}")
    print(f"{'='*40}")
    
    predictions = predict_with_probability(test_input, model, tokenizer, num_predictions=2, temperature=0.5)
    
    for i, pred in enumerate(predictions, 1):
        print(f"\nPrediction #{i}:")
        print(f"  → {pred['numbers']}")
        print(f"  Probability: {pred['probability']:.1f}%")

# Calculate overall statistics
print("\n" + "="*60)
print("PREDICTION STATISTICS")
print("="*60)
print("\nNote: Probability scores are estimates based on model confidence.")
print("For more accurate probabilities, consider:")
print("1. Training on larger historical datasets")
print("2. Using ensemble methods with multiple models")
print("3. Incorporating statistical analysis of number frequencies")
print("4. Remember: Lottery numbers are random - use predictions responsibly!")

In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)